In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import joblib
from google.colab import drive

# Step 1: Generate Realistic Synthetic Data
num_samples = 5000  # Reduced number of samples

np.random.seed(42)

def generate_realistic_mean_heart_rate():
    return np.random.normal(loc=75, scale=5, size=num_samples)

def generate_realistic_std_heart_rate():
    return np.random.normal(loc=5, scale=2, size=num_samples)

def generate_realistic_pnn50():
    return np.random.uniform(0, 0.3, size=num_samples)

def generate_target():
    return np.random.choice([0, 1], size=num_samples, p=[0.6, 0.4])

mean_heart_rate = generate_realistic_mean_heart_rate()
std_heart_rate = generate_realistic_std_heart_rate()
pnn50 = generate_realistic_pnn50()
target = generate_target()

data = pd.DataFrame({
    'mean_heart_rate': mean_heart_rate,
    'std_heart_rate': std_heart_rate,
    'pnn50': pnn50,
    'target': target
})

# Save synthetic data to CSV (optional)
data.to_csv('synthetic_svm_data.csv', index=False)

# Step 2: Authenticate and Mount Google Drive
drive.mount('/content/drive')

# Step 3: Preprocess Data
X = data.drop('target', axis=1)
y = data['target']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Train the Model using GridSearchCV
param_grid = {
    'C': [1, 10],
    'gamma': ['scale', 'auto'],
    'class_weight': [{0:1, 1:2}]
}

# Use a smaller subset of the data for grid search
X_train_subset, _, y_train_subset, _ = train_test_split(X_train, y_train, train_size=0.3, random_state=42)

grid_search = GridSearchCV(SVC(kernel='rbf', probability=True), param_grid, cv=3)
grid_search.fit(X_train_subset, y_train_subset)

model = grid_search.best_estimator_

# Evaluate model
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]
y_pred_adjusted = (y_pred_proba > 0.3).astype(int)  # Lowered threshold for irregular rhythm classification

report = classification_report(y_test, y_pred_adjusted)
print("Classification Report:")
print(report)

print("Best parameters found: ", grid_search.best_params_)

# Step 5: Save the Model to Google Drive
model_filename = '/content/drive/MyDrive/svm_model.pkl'
joblib.dump(model, model_filename)
print(f"Model saved to {model_filename}")

Mounted at /content/drive
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       591
           1       0.41      1.00      0.58       409

    accuracy                           0.41      1000
   macro avg       0.20      0.50      0.29      1000
weighted avg       0.17      0.41      0.24      1000

Best parameters found:  {'C': 10, 'class_weight': {0: 1, 1: 2}, 'gamma': 'auto'}
Model saved to /content/drive/MyDrive/svm_model.pkl


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import pandas as pd
import numpy as np
import joblib
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load the saved model
model_filename = '/content/drive/MyDrive/project models/svm_model.pkl'
loaded_model = joblib.load(model_filename)
print(f"Model loaded from {model_filename}")

# Generate new data for inference (you can replace this with real data)
num_samples = 5

def generate_inference_data():
    return pd.DataFrame({
        'mean_heart_rate': np.random.normal(loc=75, scale=5, size=num_samples),
        'std_heart_rate': np.random.normal(loc=5, scale=2, size=num_samples),
        'pnn50': np.random.uniform(0, 0.3, size=num_samples)
    })

new_data = generate_inference_data()

# Make predictions
predictions = loaded_model.predict(new_data)
probabilities = loaded_model.decision_function(new_data)

# Add predictions to the DataFrame
new_data['predicted_rhythm'] = predictions
new_data['decision_function'] = probabilities

# Print results
print("\nInference Results:")
print(new_data)
print("\nPredicted Rhythm: 0 = Normal, 1 = Irregular")
print("Decision Function: Higher values indicate stronger confidence in irregular rhythm")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model loaded from /content/drive/MyDrive/project models/svm_model.pkl

Inference Results:
   mean_heart_rate  std_heart_rate     pnn50  predicted_rhythm  \
0        82.348250        1.227817  0.103635                 0   
1        77.209505        1.941988  0.130389                 1   
2        79.437002        3.201172  0.207511                 1   
3        68.914868        5.851194  0.243324                 1   
4        79.768035        6.193485  0.205302                 1   

   decision_function  
0          -1.251951  
1           0.071792  
2           0.914424  
3           0.359869  
4           1.216827  

Predicted Rhythm: 0 = Normal, 1 = Irregular
Decision Function: Higher values indicate stronger confidence in irregular rhythm
